In [ ]:
!pip install datasets
# !pip install torchvision
# !pip install torch transformers

In [ ]:
import sys
from google.colab import drive
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import numpy as np
import warnings

# !wget -O vqgan_imagenet_f16_16384.yaml https://heibox.uni-heidelberg.de/f/140747ba53464f318b3d/?dl=1
# !wget -O vqgan_imagenet_f16_16384.ckpt https://heibox.uni-heidelberg.de/f/13e75467430e4b98a1ff/?dl=1

# Ignore specific warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
drive.mount('/content/drive')
%cd /content/drive/MyDrive/CS224N_project/
dataset = load_from_disk('/content/drive/MyDrive/CS224N_project/MagicBrush')
!git clone https://github.com/xinntao/ESRGAN.git
# %cd ESRGAN
sys.path.append('/content/drive/MyDrive/CS224N_project/')
sys.path.append('/content/drive/MyDrive/CS224N_project/ESRGAN')

In [ ]:
first_image_data = dataset['train'][3]
source_img = first_image_data['source_img']
mask_img = first_image_data['mask_img']
target_img = first_image_data['target_img']

fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(source_img)
axs[0].set_title('Source Image')
axs[0].axis('off')

axs[1].imshow(np.array(mask_img)[:, :, -1])
axs[1].set_title('Mask Image')
axs[1].axis('off')

axs[2].imshow(target_img)
axs[2].set_title('Target Image')
axs[2].axis('off')

plt.show()

print(first_image_data['instruction'])

In [ ]:
%load_ext autoreload
%autoreload 2
# from MMF import MMF
from MMF_mask import MMF_mask
from MMF_mask_CLIP import MMF_mask_CLIP
from MMF_mask_cat import MMF_mask_cat
from MMF_mask_multihead import MMF_mask_multihead
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from Preprocess_mask import MagicBrushDataset
# from MMF_transform import MultimodalTransformer
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from Image_printer import visualize_results
from PatchDiscriminator import PatchDiscriminator
from torchvision.utils import save_image
import torch.optim.lr_scheduler as lr_scheduler

batch_size = 4

dataset_train = MagicBrushDataset(dataset['train'])
dataloader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
dataset_dev = MagicBrushDataset(dataset['dev'])
dataloader_dev = DataLoader(dataset_dev, batch_size=batch_size, shuffle=True)

dataset_size = len(dataset_train)
train_size = int(0.2 * dataset_size)
eval_size = int(0.02 * dataset_size)
test_size = dataset_size - train_size - eval_size
train_dataset, eval_dataset, test_dataset = random_split(dataset_train, [train_size, eval_size, test_size])

dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dataloader_eval = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


dataset_size = len(dataset_train)
indices = np.arange(dataset_size)
np.random.shuffle(indices)
subset_indices = indices[:int(0.2 * dataset_size)]
subset_indices = list(map(int, subset_indices))

subset_dataset = Subset(dataset_train, subset_indices)
dataloader_subset = DataLoader(subset_dataset, batch_size=batch_size, shuffle=True)
subset_dataset = Subset(dataset_dev, subset_indices)
dataloader_subset_dev = DataLoader(subset_dataset, batch_size=batch_size, shuffle=True)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MMF_mask_cat().to(device)
model_attention = MMF_mask().to(device)
model_CLIP = MMF_mask_CLIP().to(device)
model_multihead = MMF_mask_multihead().to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_attention = torch.optim.Adam(model_attention.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_attention_multihead = torch.optim.Adam(model_multihead.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_CLIP = torch.optim.Adam(model_CLIP.parameters(), lr=0.0002, betas=(0.5, 0.999))
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer_attention, mode='min', factor=0.5, patience=3)
criterion = nn.BCELoss()

save_path = '/content/drive/MyDrive/CS224N_project/MMF_mask_cat.pth'########## need to change
save_path_attention = '/content/drive/MyDrive/CS224N_project/MMF_model_cross_attention.pth'
save_path_attention_multihead = '/content/drive/MyDrive/CS224N_project/MMF_model_cross_attention_multihead.pth'
save_path_CLIP = '/content/drive/MyDrive/CS224N_project/MMF_model_CLIP.pth'

# model.load_state_dict(torch.load(save_path), strict=False)
# model_attention.load_state_dict(torch.load(save_path_attention), strict=False)
# model_CLIP.load_state_dict(torch.load(save_path_CLIP), strict=False)
# MMF_mask_multihead.load_state_dict(torch.load(save_path_attention_multihead), strict=False)

In [ ]:
# scores_cat = [] ########## Cat #############
# import matplotlib.pyplot as plt
# num_epochs = 50
# for epoch in range(num_epochs):
#     model.train()
#     scores = []
#     for source_img, target_img, instruction in tqdm(dataloader_train, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
#         source_img = source_img.to(device)
#         target_img = target_img.to(device)
#         # Forward pass
#         prediction = model(source_img, instruction)
#         # prediction = (prediction > 0.9).float()
#         # Calculate loss
#         loss = criterion(prediction, target_img)
#         # Backward pass and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         scores.append(loss.item())
#     print(f'Epoch {epoch+1}/{num_epochs}, Avg Loss: {np.mean(scores):.4f}')
#     scores_cat.append(np.mean(scores))
#     torch.save(model.state_dict(), save_path)

In [ ]:
# scores_CLIP = [] ########## CLIP ###################
# import matplotlib.pyplot as plt
# num_epochs = 50
# best_score = float('inf')
# for epoch in range(num_epochs):
#     model.train()
#     scores = []
#     for source_img, target_img, instruction in tqdm(dataloader_train, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
#         source_img = source_img.to(device)
#         target_img = target_img.to(device)
#         # Forward pass
#         prediction = model_CLIP(source_img, instruction)
#         # prediction = (prediction > 0.9).float()
#         # Calculate loss
#         loss_CLIP = criterion(prediction, target_img)
#         # Backward pass and optimization
#         optimizer_CLIP.zero_grad()
#         loss_CLIP.backward()
#         optimizer_CLIP.step()
#         scores.append(loss_CLIP.item())
#     print(f'Epoch {epoch+1}/{num_epochs}, Avg Loss: {np.mean(scores):.4f}')
#     torch.save(model.state_dict(), save_path_CLIP)
#     scores_CLIP.append(np.mean(scores))

In [ ]:
# scores_attention = [] ########## Attention #############
# import matplotlib.pyplot as plt
# num_epochs = 50
# best_score = float('inf')
# for epoch in range(num_epochs):
#     model_attention.train()
#     scores = []
#     for source_img, target_img, instruction in tqdm(dataloader_train, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
#         source_img = source_img.to(device)
#         target_img = target_img.to(device)
#         # Forward pass
#         prediction = model_attention(source_img, instruction)
#         # prediction = (prediction > 0.9).float()
#         # Calculate loss
#         loss_attention = criterion(prediction, target_img)
#         # Backward pass and optimization
#         optimizer_attention.zero_grad()
#         loss_attention.backward()
#         optimizer_attention.step()
#         scores.append(loss_attention.item())
#     print(f'Epoch {epoch+1}/{num_epochs}, Avg Loss: {np.mean(scores):.4f}')
#     torch.save(model_attention.state_dict(), save_path_attention)
#     scores_attention.append(np.mean(scores))


In [ ]:
scores_attention_multihead = [] ########## Attention #############
import matplotlib.pyplot as plt
num_epochs = 50
best_score = float('inf')
for epoch in range(num_epochs):
    # MMF_mask_multihead.train()
    scores = []
    for source_img, target_img, instruction in tqdm(dataloader_train, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
        source_img = source_img.to(device)
        target_img = target_img.to(device)
        # Forward pass
        prediction = model_multihead(source_img, instruction)
        # prediction = (prediction > 0.9).float()
        # Calculate loss
        loss_attention_multihead = criterion(prediction, target_img)
        # Backward pass and optimization
        optimizer_attention_multihead.zero_grad()
        loss_attention_multihead.backward()
        optimizer_attention_multihead.step()
        scores.append(loss_attention_multihead.item())
    print(f'Epoch {epoch+1}/{num_epochs}, Avg Loss: {np.mean(scores):.4f}')
    torch.save(model_multihead.state_dict(), save_path_attention)
    scores_attention_multihead.append(np.mean(scores))

In [ ]:
model.eval()
model_attention.eval()
model_CLIP.eval()
MMF_mask_multihead.eval()
# with torch.no_grad():
#     scores = []
#     for source_img, target_img, instruction in tqdm(dataloader_eval, desc='Evaluating'):
#         source_img = source_img.to(device)
#         target_img = target_img.to(device)

#         outputs = model(source_img, instruction)
#         loss = criterion(outputs, target_img)
#         scores.append(loss.item())
#     print(f'Concatenation evaluation Avg Loss: {np.mean(scores):.4f}')
# with torch.no_grad():
#     scores = []
#     for source_img, target_img, instruction in tqdm(dataloader_eval, desc='Evaluating'):
#         source_img = source_img.to(device)
#         target_img = target_img.to(device)

#         outputs = model_attention(source_img, instruction)
#         loss = criterion(outputs, target_img)
#         scores.append(loss.item())
#     print(f'cross attention evaluation Avg Loss: {np.mean(scores):.4f}')
# with torch.no_grad():
#     scores = []
#     for source_img, target_img, instruction in tqdm(dataloader_eval, desc='Evaluating'):
#         source_img = source_img.to(device)
#         target_img = target_img.to(device)

#         outputs = model_CLIP(source_img, instruction)
#         loss = criterion(outputs, target_img)
#         scores.append(loss.item())
#     print(f'CLIP evaluation Avg Loss: {np.mean(scores):.4f}')
with torch.no_grad():
    scores = []
    for source_img, target_img, instruction in tqdm(dataloader_eval, desc='Evaluating'):
        source_img = source_img.to(device)
        target_img = target_img.to(device)

        outputs = MMF_mask_multihead(source_img, instruction)
        loss = criterion(outputs, target_img)
        scores.append(loss.item())
    print(f'CLIP evaluation Avg Loss: {np.mean(scores):.4f}')

In [ ]:

with torch.no_grad():
        for source_img, target_img, instruction in dataloader_eval:
            source_img = source_img.to(device)
            target_img = target_img.to(device)

            outputs = model(source_img, instruction)
            outputs = (outputs > 0.2).float()
            visualize_results(source_img, target_img, outputs, instruction)
            outputs_attention = model_attention(source_img, instruction)
            outputs_attention = (outputs_attention > 0.25).float()
            visualize_results(source_img, target_img, outputs_attention, instruction)
            outputs_CLIP = model_CLIP(source_img, instruction)
            outputs_CLIP = (outputs_CLIP > 0.2).float()
            outputs_attention = model_multihead(source_img, instruction)
            outputs_attention = (outputs_attention > 0.2).float()
            visualize_results(source_img, target_img, outputs_CLIP, instruction)
            break

In [ ]:
import pickle

# with open('scores_cat.pkl', 'wb') as file:########## need to change
# #     pickle.dump(scores_cat, file)########## need to change
# with open('scores_attention.pkl', 'wb') as file:########## need to change
#     pickle.dump(scores_attention, file)########## need to change
# with open('scores_CLIP.pkl', 'wb') as file:########## need to change
    # pickle.dump(scores_CLIP, file)########## need to change
with open('scores_attention_multihead.pkl', 'wb') as file:########## need to change
    pickle.dump(scores_attention_multihead, file)########## need to change

In [ ]:
# with open('scores_cat.pkl', 'rb') as file:
#     scores_cat = pickle.load(file)
# with open('scores_attention.pkl', 'rb') as file:
#     scores_attention = pickle.load(file)
# with open('scores_CLIP.pkl', 'rb') as file:
#     scores_CLIP = pickle.load(file)

import matplotlib.pyplot as plt

plt.plot(scores_attention_multihead, label = 'sssd')
plt.xlabel('epoch')
plt.ylabel('Score')
plt.legend()
plt.show()